# AstroHACK DEMO VLA

# Start Dask Cluster

In [ ]:
from menrva.client import local_client
log_params = {'log_to_term':True,'log_level':'DEBUG','log_to_file':True,'log_file':'/users/jsteeb/astrohack/examples/logs/hack_'}
worker_log_params = {'log_to_term':False,'log_level':'DEBUG','log_to_file':True,'log_file':'/users/jsteeb/astrohack/examples/logs/hack_'}

client = local_client(cores=16,memory_limit='10GB',log_params=log_params,worker_log_params=worker_log_params)
client

# Create holog_obs_description

In [ ]:
from astrohack.extract_holog import extract_holog
import numpy as np
import json

reference_antennas = np.array(['ea01','ea02','ea03','ea04','ea05','ea07','ea08','ea09','ea11','ea12','ea14','ea18','ea19','ea21','ea22','ea24','ea27'])
mapping_antennas = np.array(['ea06','ea10','ea13','ea15','ea16','ea17','ea23','ea25','ea26','ea28'])
scans = np.array([8,9,10,12,13,14,16,17,18,23,24,25,27,28,29,31,32,33,38,39,40,42,43,44,46,47,48,53,54,55,57])
holog_obs_description = {'map_0':{'ant':{}}}

for m_ant in mapping_antennas:
    holog_obs_description['map_0']['scans'] = scans
    holog_obs_description['map_0']['ant'][m_ant]=reference_antennas
    
    
holog_obs_description['ddi'] = np.arange(16)
print(json.dumps(holog_obs_description, indent=4, default=str))

# Extract Holog

In [ ]:
from astrohack.extract_holog import extract_holog
holog_mds = extract_holog(
    ms_name='/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.ms', 
    holog_obs_dict=holog_obs_description,
    data_col='CORRECTED_DATA',
    parallel=True,
    overwrite=True,
)

In [ ]:
# from astrohack._utils._dio import AstrohackHologFile
# holog_mds = AstrohackHologFile('/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.holog.zarr')
# holog_mds.open()

In [ ]:
holog_mds.summary()

In [ ]:
holog_mds['ddi_1']['map_0']['ant_ea25']

In [ ]:
import matplotlib.pyplot as plt
lm_da = holog_mds['ddi_1']['map_0']['ant_ea25'].DIRECTIONAL_COSINES

plt.figure()
plt.scatter(lm_da.isel(lm=0),lm_da.isel(lm=1))
plt.title('Antenna EA25 Pointings')
plt.xlabel('l (radians)')
plt.ylabel('m (radians)')
plt.show()


In [ ]:
from astrohack.dio import _load_point_file
point_mds = _load_point_file('/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.point.zarr')
point_mds['ant_ea04']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#%matplotlib widget
ant_name = 'ant_ea04'
lm_da = point_mds[ant_name].DIRECTIONAL_COSINES
r = np.sqrt(lm_da.isel(ra_dec=0,time=slice(5000,-1))**2 + lm_da.isel(ra_dec=1,time=slice(5000,-1))**2)

plt.figure()
r.plot()
plt.title('Antenna EA04')


ant_name = 'ant_ea25'
lm_da = point_mds[ant_name].DIRECTIONAL_COSINES
r = np.sqrt(lm_da.isel(ra_dec=0,time=slice(5000,-1))**2 + lm_da.isel(ra_dec=1,time=slice(5000,-1))**2)

plt.figure()
r.plot()
plt.title('Antenna EA25')



# Holog

In [ ]:
from astrohack.holog import holog
import numpy as np
cell_size = np.array([-0.0006442, 0.0006442])
grid_size = np.array([31, 31])
grid_interpolation_mode = 'linear' #'nearest', 'linear', 'cubic'

image_mds = holog(holog_name='/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.holog.zarr', padding_factor=50, 
      grid_size=grid_size, cell_size=cell_size, grid_interpolation_mode=grid_interpolation_mode,
      chan_average = True,
      scan_average = True,
      overwrite=True,
      phase_fit=True,
      apply_mask=True,
      to_stokes=True,
      parallel=True
     )

In [ ]:
# from astrohack._utils._dio import AstrohackImageFile
# image_mds = AstrohackImageFile('/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.image.zarr')
# image_mds.open()

In [ ]:
image_mds.summary()

In [ ]:
image_mds['ant_ea25']['ddi_1']

In [ ]:
plt.figure()
image_mds['ant_ea25']['ddi_1'].ANGLE.sel(pol='I').plot()

In [ ]:
plt.figure()
np.abs(image_mds['ant_ea25']['ddi_1'].BEAM.sel(pol='I')).plot()

In [ ]:
#np.abs(holog_mds['ant_ea25']['ddi_1'].APERTURE.sel(pol='U')).plot()

In [ ]:
import xarray as xr
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy
import matplotlib.patches as patches
import scipy.constants

ds = image_mds['ant_ea25']['ddi_1']

wavelength = scipy.constants.speed_of_light/ds.chan.values[0]
u = ds.u_prime.values*wavelength
v = ds.v_prime.values*wavelength

circle = patches.Circle((0,0), 12.5, fill=False, color='red', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.ANGLE[0,0,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])
ax.add_patch(circle)
plt.xlabel('u (m)')
plt.ylabel('v (m)')
plt.show()

# Panel

In [ ]:
from astrohack.panel import panel
ddi = None
panel_model = 'rigid'

panel_mds = panel(
    image_name='/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.image.zarr', 
    panel_model=panel_model,
    unit='mils', 
    parallel=True,
    overwrite=True
)

In [ ]:
# from astrohack._utils._dio import AstrohackPanelFile
# panel_mds = AstrohackPanelFile('/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.panel.zarr')
# panel_mds.open()

In [ ]:
panel_mds.summary()

In [ ]:
panel_mds['ant_ea25']['ddi_0']

In [ ]:
plt.figure()
panel_mds['ant_ea25']['ddi_0']['CORRECTIONS'].plot()

In [ ]:
from IPython.display import Image
Image(filename = "/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.panel.zarr/ant_ea25/ddi_1/surface.png", width=1500, height=1500)

In [ ]:
f = open('/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.panel.zarr/ant_ea25/ddi_1/screws.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

In [ ]:
from casacore import tables as ctables
import os
import numpy as np

ctb = ctables.table(
    os.path.join('/.lustre/cv/projects/casa/holo/demo/ea25/after_THOL0001.sb42608447.eb42709773.59842.053038750004.ms', "ANTENNA"),
    readonly=True,
    lockoptions={"option": "usernoread"},
    ack=False,
)

ant_names = ctb.getcol("NAME")
ant_id = np.arange(len(ant_names))